In [12]:
from pathlib import Path

import numpy as np
import pandas as pd
from authorship_tool.util.ml import trainer
from authorship_tool.util.ml.model import (
    LGBMSource,
    Score,
    TrainingResult,
    dump,
    pred_crosstab_normal,
)
from authorship_tool.util.path_util import PathUtil
from numpy.typing import NDArray

In [13]:
np.seterr(divide="call")
np.geterr()
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [14]:
dataset_path: Path = PathUtil.MANUAL_DATASET

In [15]:
dataset = pd.read_csv(dataset_path, index_col=0)

In [16]:
display(dataset.head(10))

,v1 sentences per paragraph,v2 words per paragraph,v3 close parenthesis present,v4 dash present,v5 semi-colon or colon present,v6 question mark present,v7 apostrophe present,v8 standard deviation of sentence length,v9 length difference for consecutive sentences,v10 sentence with < 11 words,v11 sentence with > 34 words,v12 contains although,v13 contains however,v14 contains but,v15 contains because,v16 contains this,v17 contains others or researchers,v18 contains numbers,v19 contains 2 times more capitals than period,v20 contains et,word variation,average token length,non alphabetic characters frequency,uncommon word frequency,non-alphabetic characters frequency,numeric value frequency,$,'',(,),",",.,:,CC,CD,DT,EX,FW,IN,JJ,JJR,JJS,JJ_pp,MD,NN,NNP,NNPS,NNS,PDT,POS,PRP,PRP$,RB,RBR,RBS,RP,SYM,TO,UH,VB,VBD,VBG,VBN,VBP,VBZ,WDT,WP,WP$,WRB,``,answer
0,1,14,False,False,False,False,False,0.000000,0.000000,False,False,False,False,False,False,False,False,True,True,False,0.857143,2.857143,0.571429,0.857143,0.571429,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.142857,0.000000,0.071429,0.071429,0.142857,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.0,0.000000,0.071429,0.357143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,True
1,2,8,False,False,False,False,False,2.000000,4.000000,True,False,False,False,False,False,False,False,False,True,False,1.000000,3.625000,0.125000,1.000000,0.125000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.625000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,True
2,5,164,False,False,True,False,False,12.983066,13.250000,False,True,False,False,False,False,False,False,False,False,False,0.573171,4.103659,0.091463,0.530488,0.091463,0.0,0.0,0.0,0.000000,0.000000,0.042683,0.030488,0.018293,0.048780,0.006098,0.134146,0.000000,0.006098,0.176829,0.073171,0.006098,0.000000,0.0,0.000000,0.164634,0.012195,0.000000,0.036585,0.000000,0.000000,0.060976,0.000000,0.036585,0.000000,0.000000,0.000000,0.0,0.024390,0.0,0.006098,0.042683,0.018293,0.012195,0.012195,0.024390,0.006098,0.0,0.000000,0.000000,0.0,True
3,6,149,False,True,True,False,False,4.669642,6.200000,False,False,False,False,True,False,True,False,False,False,False,0.637584,4.389262,0.120805,0.590604,0.120805,0.0,0.0,0.0,0.000000,0.000000,0.067114,0.040268,0.013423,0.013423,0.013423,0.120805,0.000000,0.000000,0.100671,0.053691,0.000000,0.006711,0.0,0.006711,0.127517,0.013423,0.000000,0.046980,0.013423,0.000000,0.046980,0.013423,0.060403,0.000000,0.000000,0.000000,0.0,0.020134,0.0,0.020134,0.087248,0.006711,0.067114,0.013423,0.000000,0.026846,0.0,0.000000,0.000000,0.0,True
4,8,294,True,False,True,False,False,18.699933,22.857143,False,True,False,True,True,False,False,False,True,False,False,0.544218,4.125850,0.136054,0.574830,0.136054,0.0,0.0,0.0,0.003401,0.003401,0.085034,0.027211,0.003401,0.037415,0.010204,0.088435,0.003401,0.000000,0.115646,0.105442,0.000000,0.000000,0.0,0.003401,0.122449,0.010204,0.003401,0.034014,0.003401,0.000000,0.064626,0.020408,0.091837,0.000000,0.003401,0.003401,0.0,0.017007,0.0,0.023810,0.068027,0.006803,0.030612,0.003401,0.000000,0.003401,0.0,0.003401,0.000000,0.0,True
5,5,91,False,True,False,False,False,13.511477,15.250000,True,True,False,False,False,False,True,False,False,False,False,0.758242,4.637363,0.175824,0.593407,0.175824,0.0,0.0,0.0,0.000000,0.000000,0.032967,0.043956,0.000000,0.021978,0.000000,0.076923,0.000000,0.000000,0.131868,0.054945,0.000000,0.000000,0.0,0.010989,0.142857,0.054945,0.000000,0.065934,0.000000,0.000000,0.054945,0.021978,0.054945,0.000000,0.000000,0.000000,0.0,0.054945,0.0,0.054945,0.010989,0.032967,0.010989,

In [17]:
print(dataset.shape)

(5249, 71)


In [18]:
print(dataset.dtypes)

v1 sentences per paragraph                          int64
v2 words per paragraph                              int64
v3 close parenthesis present                         bool
v4 dash present                                      bool
v5 semi-colon or colon present                       bool
v6 question mark present                             bool
v7 apostrophe present                                bool
v8 standard deviation of sentence length          float64
v9 length difference for consecutive sentences    float64
v10 sentence with < 11 words                         bool
v11 sentence with > 34 words                         bool
v12 contains although                                bool
v13 contains however                                 bool
v14 contains but                                     bool
v15 contains because                                 bool
v16 contains this                                    bool
v17 contains others or researchers                   bool
v18 contains n

In [19]:
print(dataset.isna().sum())

v1 sentences per paragraph                        0
v2 words per paragraph                            0
v3 close parenthesis present                      0
v4 dash present                                   0
v5 semi-colon or colon present                    0
v6 question mark present                          0
v7 apostrophe present                             0
v8 standard deviation of sentence length          0
v9 length difference for consecutive sentences    0
v10 sentence with < 11 words                      0
v11 sentence with > 34 words                      0
v12 contains although                             0
v13 contains however                              0
v14 contains but                                  0
v15 contains because                              0
v16 contains this                                 0
v17 contains others or researchers                0
v18 contains numbers                              0
v19 contains 2 times more capitals than period    0
v20 contains

In [20]:
df: pd.DataFrame = dataset.iloc[:, :-1]
nd_category: NDArray = np.array(dataset.iloc[:, -1], dtype=bool)

In [21]:
source: LGBMSource = LGBMSource(feature_data_frame=df, nd_category=nd_category)
result: TrainingResult = trainer.train_once(source)

[LightGBM] [Info] Number of positive: 3038, number of negative: 898
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001113 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9574
[LightGBM] [Info] Number of data points in the train set: 3936, number of used features: 66
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.771850 -> initscore=1.218785
[LightGBM] [Info] Start training from score 1.218785


LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


In [22]:
score: Score | None = result.score

In [23]:
if score is not None:
    print(f"auc-roc score: {score.auc_roc_score}")
    print(f"f1 score: {score.f1_score}")
    print(f"accuracy score: {score.accuracy_score}")

auc-roc score: 0.8799247256105662
f1 score: 0.9187675070028012
accuracy score: 0.8674790555978675


In [24]:
display(pred_crosstab_normal(result))

predicted,False,True
actual,,
False,155,141
True,33,984


In [25]:
dump(result, "manual")